# Serial killers with the highest known victim count

In [1]:
#Importar las librerías que nos ayudarán a correr el código. 

from bs4 import BeautifulSoup #librería para hacer parse de documentos HTML
import requests #librería para hacer los requests a la página que se va a scrappear 
import pandas as pd #librería para análisis de datos 
import re #librería para limpieza de la información 
import datanews #API 
import urllib.parse #librería para hacer el encoding de los url 
import json #librería para parsing
import time #librería para dar un intervalo de tiempo dentro de un loop
import numpy as np #librería para trabajar con arrays

### Web Scrapping de Wikipedia. 

In [2]:
#Dar la instrucción de scrappear el contenido de la página de Wikipedia donde vienen la lista de los asesinos
#seriales rankeados por número de víctimas. Para ordenar la información y que sea más legible usamos Beautiful Soup. 

wiki = requests.get("https://en.wikipedia.org/wiki/List_of_serial_killers_by_number_of_victims").content
soup = BeautifulSoup(wiki, 'html5lib')

#De todo el contenido de la página, solo me interesa la tabla donde viene agrupada la info que necesito 
#Con Inspect identifiqué que la class que necesito llamar es "wikitable sortable"

killers_info = soup.select('table[class="wikitable sortable"]')

#La información seleccionada es limpiada. 

clean_killers_info = [th.text.strip().replace("\xa0",",").split('\n\n\n') for th in killers_info]

#De la clase que identificamos, solo necesitamos la primera tabla que corresponde a los asesinos seriales 
#con mayor número de víctimas, por lo que iteramos, la seleccionamos y asignamos a una variable nueva 

killer_chart = clean_killers_info[0]


In [3]:
#Es necesario hacer una segunda limpieza para quitar los caracteres que ensucian la información y después creamos 
#una lista para guardar la información.

clean_killer_chart = [x.strip().replace(",", " ").split("\n\n") for x in killer_chart]

new_df=[]
for row in clean_killer_chart:
    new_row=[]
    for element in row:
        new_row.append(re.sub(r'\[\d*]','',element))
    new_df.append(new_row)

#Una vez que la información ya está limpia, utilizamos pandas para crear un dataframe
#Indicamos cuáles queremos que sean los nombres de las columnas 
#Eliminamos filas que no hacen sentido con la tabla y que no tienen información relevante 

df_killers = pd.DataFrame(new_df, columns = new_df[0]).drop([0])
df_killers


,Name,Country,Years active,Proven victims,Possible victims,Notes
1,Luis Garavito,Colombia Ecuador Venezuela,1992 to 1999,138,172–300+,Child-murderer torture-killer and rapist kno...
2,Pedro López,Colombia Peru Ecuador,1969 to 1979,110,300+,"Child-murderer and rapist known as ""The Monst..."
3,Javed Iqbal,Pakistan,1998 to 1999,100,100,"Child-murderer and rapist known as ""Kukri"" I..."
4,Mikhail Popkov,Russia,1992 to 2010,83,83+,"Serial rapist-killer nicknamed ""The Werewolf"" ..."
5,Daniel Camargo Barbosa,Colombia Ecuador Brazil (alleged),1974 to 1986,72,180,Child and woman murderer believed to have pos...
6,Pedro Rodrigues Filho,Brazil,1967 to 2003,71,100+,He claimed to have killed over 100 victims 47...
7,Kampatimar Shankariya,India,1977 to 1978,70,70+,This mysterious Indian serial killer used a ha...
8,Yang Xinhai,China,2000 to 2003,67,67,"Known as the ""Monster Killer"". Would enter vic..."
9,Andrei Chikatilo,Soviet Union,1978 to 1990,53,56,"Known as ""The Butcher of Rostov"" ""The Red Rip..."
10,Anatoly Onoprienko,Soviet Union Ukraine,1989 to 1996,52,52+,"Known as ""The Beast of Ukraine"" ""The Terminat..."


In [4]:
#La fila 31 es la única que sale mal.
#Nos damos cuenta que falta separar por columna '\n' en la columna "Name" que es donde se quedó toda la info
#lo asignamos a una variable para después usarla para asignar un valor 

row_31 = df_killers.loc[31]['Name'].split('\n') 

#Una vez que ya tenemos la info separada en una lista, iteramos sobre cada columna e índice 
#para asignar cada valor que le corresponde según la lista que creamos arriba. 

for cell,column in enumerate(df_killers.columns):
    df_killers.loc[31][column] = row_31[cell]


### Llamada a la API de "Data News" y consolidación de la información.

In [5]:
#Por el nombre de cada asesino hacemos un request para almacenar el URL con la noticia relacionada al asesino.
#Si no se encuentra una noticia para el asesino, asignamos "No news found". 
#Usamos la librería urllib para el encoding de URL.
#Guardamos la información en un diccionario.

news = {}

for name in df_killers['Name']:
    url = f'http://api.datanews.io/v1/news?q=%22{urllib.parse.quote(name)}%22'
    info = requests.get(url, headers = {'x-api-key': '04rolxy69c1vzr6wi5xplfqyf'})
    if info.json()['numResults'] != 0:
        news[name]=(info.json()['hits'][0]['url'])
    else:
        news[name]='No news found'
        time.sleep(2)

        
#La información que guardamos en el diccionario la pasamos a un dataframe para visualización. 
df_news = pd.DataFrame(news, index=[0]).T.rename(columns={0: 'url'})

#Unimos el dataframe que contiene la información de los asesinos seriales con la que contiene el url de la 
#noticia por asesino para consolidar los resultados.Tenemos que hacer que coincidan en el índice, por lo que en 
#df_killers pasamos la columna "Name" como el índice y posterior hacemos el join. 

df_killers = df_killers.set_index(df_killers['Name'], drop=False).drop('Name',axis=1)
final_df = df_killers.join(df_news)

#Finalmente imprimimos el resultado 
final_df

,Country,Years active,Proven victims,Possible victims,Notes,url
Name,,,,,,
Luis Garavito,Colombia Ecuador Venezuela,1992 to 1999,138,172–300+,Child-murderer torture-killer and rapist kno...,https://www.vice.com/en_us/article/bv8bv4/the-...
Pedro López,Colombia Peru Ecuador,1969 to 1979,110,300+,"Child-murderer and rapist known as ""The Monst...",https://www.heraldo.es/noticias/deportes/futbo...
Javed Iqbal,Pakistan,1998 to 1999,100,100,"Child-murderer and rapist known as ""Kukri"" I...",https://www.urdupoint.com/en/pakistan/death-an...
Mikhail Popkov,Russia,1992 to 2010,83,83+,"Serial rapist-killer nicknamed ""The Werewolf"" ...",https://www.thesun.co.uk/news/12463599/serial-...
Daniel Camargo Barbosa,Colombia Ecuador Brazil (alleged),1974 to 1986,72,180,Child and woman murderer believed to have pos...,https://www.vice.com/en_us/article/bv8bv4/the-...
Pedro Rodrigues Filho,Brazil,1967 to 2003,71,100+,He claimed to have killed over 100 victims 47...,https://economia.uol.com.br/noticias/redacao/2...
Kampatimar Shankariya,India,1977 to 1978,70,70+,This mysterious Indian serial killer used a ha...,No news found
Yang Xinhai,China,2000 to 2003,67,67,"Known as the ""Monster Killer"". Would enter vic...",No news found
Andrei Chikatilo,Soviet Union,1978 to 1990,53,56,"Known as ""The Butcher of Rostov"" ""The Red Rip...",https://www.newshub.co.nz/home/lifestyle/2020/...


### Save final data to CSV.

In [11]:
#Guardar la información en un CSV para análises posteriores. 

final_df.to_csv('serial_killers.csv', index = False)